In [1]:
#! pip3 install xlrd >= 1.0.0
#! pip3 install openpyxl
#! pip3 install pattern
#!pip install JapaneseTokenizer # dependency needs to be installed mecab_wrapper 
#!pip install tinysegmenter
#! git clone https://github.com/stopwords-iso/stopwords-ja.git

In [2]:
#load module
import pandas as pd
import re
import nltk
import codecs
import string
from pattern.en import parsetree
from pattern.en import parse
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
#load data
data = pd.read_excel('Unstructured Data Japanese.xls', encoding='utf-8', index_col=0)
data.head()

,Kraft Super bowl data,English
Sno,,
1,「私は、私は再び8だように行動し、Velveetaマックとディナーにチーズを食べたいです。し...,I want to act like I'm 8 again and eat cheese ...
2,this_Ray私は病気で寝ていると私は眠ることができません。 「赤穂PO SIトラヴィスク...,"this_Ray I'm sick in bed and I can't sleep. ""A..."
3,RT JeffSchultzAJC：グッデルは、彼はまた、クラを掃除された理由のAFCタイト...,RT JeffSchultzAJC: Goodell defends being at Cr...
4,「グッデルは、彼はまた、クラフトのガレージ、プールを掃除された理由のAFCタイトルゲームはま...,Goodell defends being at Craft House night bef...
5,RTのc8suchy：「WorldStarComedy：このSMOKE TORNADO CR...,"RT c8suchy: ""WorldStar Comedy: This SMOKE TORN..."


In [4]:
print(data.columns)
data.drop(['English'],axis=1,inplace=True)
print(data.columns)

Index(['Kraft Super bowl data', 'English'], dtype='object')
Index(['Kraft Super bowl data'], dtype='object')


#### some japanese text translated to english samples.

I want to act like I'm 8 again and eat cheese for Velveeta Mack and dinner. But it really isn't cardio worth I'll have to do tomorrow

this_Ray I'm sick in bed and I can't sleep. "Ako PO SI Travis Craft" pops in my head and I thought of you. I want to do adobo now...

RT JeffSchultzAJC: Goodell defends being at Craft House night before the AFC title game why he was also cleaning Kula has not yet dealt with...

Goodell defends being at Craft House night before the AFC title game has yet to deal with why he was also cleaning the craft garage and pool.

RT c8suchy: "WorldStar Comedy: This SMOKE TORNADO CRAZY AFedU + 00A0U + 00BDedU + 00B2U + 00AFedU + 00A0U + 00BDedU + 00B8U + 0082edU + 00A0U + 00BDedU + 00B8U + 0082 http://t.co/90vHMsnnnheez

Another 30 minutes before my alarm goes off, the window glass detergent is hitting my window EDU+00A0U+00BDedU+00B8U+00A1edU+00A0U+00BDedU+00B8U+00A1 #boreoff U+270B

............All natural crispy craft peanut butter is like a crack at alledU+00A0U+00BDedU+00B8U+008D

La...... Enjoy Goodell/Craft. The Fall of America-No boring name restrictions, entertainment or immorality

I eat unhealthy pizza quantities and craft dinners

Craft Mac and cheese have a ridiculous amount of protein in it because of what it is.

RealSkipBayless Brady too cheats... and Goodell Craft... has done a zero dredibility fake analyst figuring out how to cover it...
snooddood7 I want all the analysts who yelled at the swindler to provide their pinky with a white napkin folded into a craft.
One place for drawing our fifty: Antidote to be boring! Check out our timeline short links & # 038. Input code 5yzAw 2 Billing aurora_kraft
Weei Renesas cprice NFL craft seeking leaky cuz apology to media and criticism. Do not invest for itself. "It's my job" Doesn't apply here.
RT ActuallyDontGAS: Kraft bkravitz sought you a FKN incompetent leak-free apology for investigation. You can't even get it RIGH...
For the craft bkravitz survey you asked for an FKN sloppy leak-free apology. If you try, you can't even get it right.
BostonGlobe fine is forced by craft demand for an apology that never flies... but a leak apology is appropriate
It's important to make this distinction MikeReiss: Craft seeks apology for leaks, etc. NFL survey seeks apology for hes
bkravitz craft wants all leak cuz apologies from the NFL. NFL should apologize for complicating with witch hunting makin
RT shalisemyoung: Goodell ... asked about craft saying NE should get an apology if ball blame is false and said it was his job to protect INTE...
Goodell said it's his job to protect the integrity of the game... and asked about craft saying NE should get an apology if the ball blame is false
I honestly misunderstood Robert Kraft for craft cheese and I wondered why craft cheese would apologize for the alleged soccer claim.
RT Jabba_Jabba_Jaw: Hot craft singles in your area open up the value of effort and are not really difficult.
RT Jabba_Jabba_Jaw: Hot craft singles in your area open up the value of effort and are not really difficult.


In [5]:
data.rename(columns={'Kraft Super bowl data': 'text'}, inplace=True)
data.head(5)

,text
Sno,
1,「私は、私は再び8だように行動し、Velveetaマックとディナーにチーズを食べたいです。し...
2,this_Ray私は病気で寝ていると私は眠ることができません。 「赤穂PO SIトラヴィスク...
3,RT JeffSchultzAJC：グッデルは、彼はまた、クラを掃除された理由のAFCタイト...
4,「グッデルは、彼はまた、クラフトのガレージ、プールを掃除された理由のAFCタイトルゲームはま...
5,RTのc8suchy：「WorldStarComedy：このSMOKE TORNADO CR...


In [6]:
ja_stopwords = []

with open('stopwords-ja/raw/japanese-stopwords.txt', encoding="utf8",mode='r') as f:
    stopwords_tmp = f.readlines()
ja_stopwords.extend(stopwords_tmp)
with open('stopwords-ja/raw/gh-stopwords-json-ja.txt', encoding="utf8",mode='r') as f:
    stopwords_tmp = f.readlines()
ja_stopwords.extend(stopwords_tmp)
with open('stopwords-ja/raw/ranksnl-japanese.txt', encoding="utf8",mode='r') as f:
    stopwords_tmp = f.readlines()
ja_stopwords.extend(stopwords_tmp)
with open('stopwords-ja/raw/bbalet_stopwords_ja.txt', encoding="utf8",mode='r') as f:
    stopwords_tmp = f.readlines()
ja_stopwords.extend(stopwords_tmp)

ja_stopwords = [i.strip('\n').strip('\t').strip(' ') for i in ja_stopwords]

ja_stopwords.extend(['「','」','：'])

print(ja_stopwords[:10])
print(ja_stopwords[-10:])

['これ', 'それ', 'あれ', 'この', 'その', 'あの', 'ここ', 'そこ', 'あそこ', 'こちら']
['ほとんど', 'と共に', 'といった', 'です', 'とも', 'ところ', 'ここ', '「', '」', '：']


In [7]:
#string.punctuation
# japanese puntuations
ja_punctuations = """’！ "＃$％＆\ '（）* +、-。/ :; <=>？@" \\ "^ _` {|}〜'!"""+string.punctuation
en_alphabets = """QWERTYUIOPASDFGHJKLZXCVBNMqwertyuiopasdfghjklzxcvbnm"""

In [8]:
import tinysegmenter
segmenter = tinysegmenter.TinySegmenter()
tokens = segmenter.tokenize(u"10日放送の「中居正広のミになる図書館」（テレビ朝日系）で、SMAPの中居正広が、篠原信一の過去の勘違いを明かす一幕があった。")
print(tokens)
tokens_without_stopwords = [i for i in tokens if i not in ja_stopwords and i not in ja_punctuations and not i.isdigit()]
print(tokens_without_stopwords)

['1', '0', '日', '放送', 'の', '「', '中居', '正広', 'の', 'ミ', 'に', 'なる', '図書館', '」', '（', 'テレビ', '朝日', '系', '）', 'で', '、', 'SMAP', 'の', '中居', '正広', 'が', '、', '篠原', '信一', 'の', '過去', 'の', '勘違い', 'を', '明かす', '一幕', 'が', 'あっ', 'た', '。']
['日', '放送', '中居', '正広', 'ミ', '図書館', 'テレビ', '朝日', '系', 'SMAP', '中居', '正広', '篠原', '信一', '過去', '勘違い', '明かす', '一幕']


#### Above text and it's tokenizations translation

#Masahiro Nakai of SMAP revealed a misunderstanding of Shin'ichi Shinohara's past misunderstandings on the 
#10th broadcast "Masahiro Nakai's Mi no Mi Naru Library" (TV Asahi).
#--------- tokens
#['1', '0','day','broadcast','no',''',' Nakai',' Masahiro','no',' Mi',' become',' become',' Library',''',' (',
#'TV','Asahi','series',')','de',',','SMAP','no','Nakai','Masahiro' ,'Ga',',','shinohara','shinichi','no','past',
#'no','misunderstanding','reveal','one act','ga', 'there were', '. ']
#--------- toekns without stopwords
#['1', '0',' Sun',' Broadcast',''',' Nakai',' Masahiro',' Mi',' Library',''',' (',' TV', ' Asahi','series',')',',',
#'SMAP','Nakai','Masahiro',',','Shinohara','Shinichi','past','misunderstanding','reveal' "," one act ",". ']
#--------- tokens without stopwords, digits, punctuations
#['Sun','Broadcast','Nakai','Masahiro','Mi','Library','Television','Asahi','Affiliation','SMAP','Nakai',
#'Masahiro',' Shinohara','Shinichi','Past','Misunderstanding','Clear','One act']

In [9]:
def ja_tokenize_and_cleanse(text):
    #replace strings between brackets and parenthesis with blank
    text = re.sub(r'\(.*?\)', '', text)
    text = re.sub(r'\[.*?\]', '', text)
    
    # replace repeated characters with single
    text = re.sub('\.+', '.', text)
    
    # replace urls with blank
    text = re.sub(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', '', text)
    
    tokens = segmenter.tokenize(u"{}".format(text))
    tokens = [i.strip() for i in tokens]
    tokens_without_stopwords = [i for i in tokens if i not in ja_stopwords and i not in ja_punctuations and not i.isdigit() and i not in en_alphabets]
    
    return tokens_without_stopwords

In [10]:
text = """.........すべての自然なカリカリクラフトピーナッツバターはalledU + 00A0U + 00BDedU + 00B8U + 008Dで亀裂のようなものです"""
print(ja_tokenize_and_cleanse(text))

['.すべて', '自然', 'カリカリクラフトピーナッツバター', 'alledU', 'BDedU', '亀裂', 'なもの']


### analyse overview of data

#### Reference Data , that needs to be taken care.
    「私は、私は再び8だように行動し、Velveetaマックとディナーにチーズを食べたいです。しかし、それは本当に私は明日行う必要があるでしょう心肺価値はありません ![image.png](attachment:image.png)
    this_Ray私は病気で寝ていると私は眠ることができません。 「赤穂PO SIトラヴィスクラフトは、「私の頭の中にポップと私はあなたのことを考えました。私は今アドーボーをしたいです... ![image.png](attachment:image.png)
    RT JeffSchultzAJC：グッデルは、彼はまた、クラを掃除された理由のAFCタイトルゲームはまだ対処していない前に、クラフトの家の夜にいる擁護します... ![image.png](attachment:image.png)
    「グッデルは、彼はまた、クラフトのガレージ、プールを掃除された理由のAFCタイトルゲームはまだ対処していない前に、クラフトの家の夜にいる擁護しています。![image.png](attachment:image.png)
    RTのc8suchy：「WorldStarComedy：このSMOKE TORNADO CRAZY AFedU + 00A0U + 00BDedU + 00B2U + 00AFedU + 00A0U + 00BDedU + 00B8U + 0082edU + 00A0U + 00BDedU + 00B8U + 0082 http://t.co/90vHMnnHUz"tayy_shee18 ![image.png](attachment:image.png)
    ............すべての自然なカリカリクラフトピーナッツバターはalledU + 00A0U + 00BDedU + 00B8U + 008Dで亀裂のようなものです ![image.png](attachment:image.png)
    「私は不健康なピザの量とクラフトの夕食を食べます ![image.png](attachment:image.png)
    「クラフトカナダ：新しい印刷用クーポンが利用可能！シェイクN '焼く澪とジェロhttp://t.co/7bErQm74Amに保存 ![image.png](attachment:image.png)


#### Observations:
    data has 
    1. http urls, that needs to be removed.
    2. image attachment name or string needs to be removed
    3. don't have much knowledge on this language. language understanding can help to explore more on this. for now it's blank.

In [11]:
data['cleansed_tokens'] = data['text'].apply(lambda x: ja_tokenize_and_cleanse(x))

In [12]:
for rec in data[:5].iterrows():
    print(rec[1]['text']," --- ",rec[1]['cleansed_tokens']," --- ")
    print()

「私は、私は再び8だように行動し、Velveetaマックとディナーにチーズを食べたいです。しかし、それは本当に私は明日行う必要があるでしょう心肺価値はありません  ---  ['再び', '行動', 'Velveeta', 'マック', 'ディナー', 'チーズ', '食べ', 'たい', '本当', '明日', '行う', '必要', 'でしょ', '心肺', '価値', 'ませ']  --- 

this_Ray私は病気で寝ていると私は眠ることができません。 「赤穂PO SIトラヴィスクラフトは、「私の頭の中にポップと私はあなたのことを考えました。私は今アドーボーをしたいです...  ---  ['this', 'Ray', '病気', '寝', '眠る', 'ませ', '赤穂', 'PO', 'SI', 'トラヴィスクラフト', '頭', '中', 'ポップ', 'あなた', '考え', 'まし', '今', 'アドーボー', 'たい']  --- 

RT JeffSchultzAJC：グッデルは、彼はまた、クラを掃除された理由のAFCタイトルゲームはまだ対処していない前に、クラフトの家の夜にいる擁護します...  ---  ['JeffSchultzAJC', 'グッデル', 'クラ', '掃除', '理由', 'AFC', 'タイトルゲーム', 'まだ', '対処', '前', 'クラフト', '家', '夜', '擁護']  --- 

「グッデルは、彼はまた、クラフトのガレージ、プールを掃除された理由のAFCタイトルゲームはまだ対処していない前に、クラフトの家の夜にいる擁護しています。  ---  ['グッデル', 'クラフト', 'ガレージ', 'プール', '掃除', '理由', 'AFC', 'タイトルゲーム', 'まだ', '対処', '前', 'クラフト', '家', '夜', '擁護']  --- 

RTのc8suchy：「WorldStarComedy：このSMOKE TORNADO CRAZY AFedU + 00A0U + 00BDedU + 00B2U + 00AFedU + 00A0U + 00BDedU + 00B8U + 0082edU + 00A0U + 00BDedU + 00B8U + 0082 http://t.c

In [13]:
from collections import Counter

terms_all = [word for wordlist in data['cleansed_tokens'] for word in wordlist]
count_all = Counter()
# Update the counter
count_all.update(terms_all)
# Print the first 5 most frequent words --> meaningful phrases
print(count_all.most_common(50))
#print([i[0] for i in count_all.most_common(250)])

[('ウ', 144), ('Kラft', 127), ('クラフト', 98), ('ロベrtKラft', 67), ('ドン', 67), ('ビll', 67), ('スペrボwlXLイX', 67), ('メss', 66), ('Wイth', 66), ('RTムシc', 66), ('tヘ', 65), ('オ/GvHSp', 64), ('ア', 55), ('あなた', 54), ('チーズ', 52), ('ませ', 52), ('ト', 45), ('まし', 40), ('エd', 39), ('オン', 38), ('オ', 35), ('アンp', 34), ('BDedU', 31), ('fオr', 31), ('必要', 30), ('アンd', 30), ('グッデル', 25), ('レcイペs', 24), ('イン', 22), ('オオキングpゴオd', 22), ('Velveeta', 21), ('すべて', 21), ('velveeta', 21), ('エアr', 21), ('謝罪', 20), ('\u200b', 20), ('イs', 20), ('ヨウ', 19), ('BD', 19), ('たい', 18), ('イt', 18), ('持っ', 17), ('NFL', 17), ('場合', 17), ('fロン', 17), ('ヂンネr', 17), ('オウt', 16), ('tハt', 16), ('bkravitz', 15), ('多く', 15)]


### English translated top words and phrases.

['U','K Ra ft','Craft','Robe rt K Ra ft','Don','Bill','Super Bo wlXL Lee X','Me ss','W th' ,'RT MUSHI c','t HE','O/GvHSp','A','You','Cheese','Not','TO','Better','E d','ON'. ,'O','Ann p','BDedU','f o r','Need','Ann d','Goodell','Le c ipe s','In','Oh king p Goo d' ,'Velveeta','all','velveeta','air r','apology','\u200b','i','you','BD','tai','it',' Have','nfl','case','fron','dine r','oh t','t ha t','bkravitz','more','wa tch','video',' 'T','l ike','heese','craft cheese','love rt','oh f','you r','...','edU','make','shit',' Kraft','They','Pat Rio ts','Goode ll','Ne d','Page','Ho w','Masuka','EDU','Bobcraft','Visit' ','De l c sulfur s','idea s','i','pe ch','id d','bane g','previous','angry','just',' t hi s','hi s','sw','more','oo ds','un f eye rly','attack d','dinner','via','like',' k la ft','ha s','yun y','ne w','wa lma rt','do','middle','false','work','a','nata ',' game','・','target','print','shell','do','n y','e ps','bamboo s','video s','mac' ,'w ll','eat','method','survey','get','say']

In [14]:
# most frequently occuring terms
countVectorizer = CountVectorizer() 
countVector = countVectorizer.fit_transform([' '.join(text) for text in data['cleansed_tokens']])
print('{} Number of tweets has {} words'.format(countVector.shape[0], countVector.shape[1]))
#print(countVectorizer.get_feature_names())

count_vect_df = pd.DataFrame(countVector.toarray(), columns=countVectorizer.get_feature_names())
count_vect_df.head()

541 Number of tweets has 2671 words


,1ン,abby,about,ac,actuallydontgas,aedu,afc,afcc,afedu,aint,...,飛ぶ,食べ,食べる,食べ物,食事,食品,養う,高価,魔女,麻疹
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# Term-Document Matrix
term_doc_df = count_vect_df.transpose()
term_doc_df.head()

,0,1,2,3,4,5,6,7,8,9,...,531,532,533,534,535,536,537,538,539,540
1ン,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abby,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
about,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ac,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
actuallydontgas,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
term_doc_df.index.name='Token'

In [17]:
# Dump as excel
term_doc_df[:500].to_excel("tfidf-output-japanese.xlsx") 

In [18]:
import openpyxl 
from openpyxl.styles import Font 

#wb = openpyxl.Workbook() 
wb = openpyxl.load_workbook("tfidf-output-japanese.xlsx")
sheet = wb.active 

sheet.insert_rows(0,1)

# merge cell.
sheet.merge_cells('B1:Z1') 

sheet.cell(row = 1, column = 2).value = 'S.No.(Unique Identifier of a comment.)'

# set the font style to bold 
sheet.cell(row = 1, column = 2).font = Font(size = 24, bold = True) 

wb.save('tfidf-output-merge-japanese.xlsx')

#### Summary of Exploratory Data Analysis

    - these are general texts. still texts are quite unclean. some taken care here. still more required to be handled such as spell checker, more accurate phrase extracter using rule based parser or using NER. but langauge knowledge would have helped much.
    - used Google translator to validate result. even some results have pasted to compare.
    - due to lack of memory(3GB RAM) in my system, i'm dumping few samples of term-frequency records to excel. user can use same code to dump all records with having good memory system. or we can do other ways like using DASK, or writing file batch by batch.
    - have mentioned comments with each code. i'm sure those are understandable. so not writing much here.
    - with that 4 questions regarding to this task have been answered through code, comments, and image of results.